# System Dependencies and Additional Packages
This cell updates the system packages and installs ffmpeg for audio processing, along with additional Python packages including datasets, transformers, torchaudio, evaluate, jiwer, torchcodec, and tensorboard. It also upgrades transformers and accelerate to their latest versions.

In [10]:

!apt-get update -y
!apt-get install -y ffmpeg
!pip install datasets transformers torchaudio evaluate jiwer torchcodec audiomentations tensorboard scikit-learn accelerate
!pip install --upgrade transformers accelerate 


Hit:1 http://archive.ubuntu.com/ubuntu noble InRelease
Hit:2 http://archive.ubuntu.com/ubuntu noble-updates InRelease                 
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu noble-backports InRelease               
Hit:5 http://security.ubuntu.com/ubuntu noble-security InRelease               
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu noble InRelease   
Reading package lists... Done
W: https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:6.1.1-3ubuntu5).
0 upgraded, 0 newly installed, 0 to remove and 63 not upgraded.


# Imports and Configuration Setup
This cell imports all necessary libraries for data processing, audio handling, and machine learning. It sets up environment variables to disable tokenizers parallelism warnings and defines configuration parameters including the number of processors and sampling rate. It also creates the base directory for the training data.

In [11]:
import os
import subprocess
import tarfile
from pathlib import Path
from tqdm import tqdm
import pandas as pd
from datasets import load_dataset, Features, Value, Audio
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate
from concurrent.futures import ProcessPoolExecutor, as_completed

# Set environment variable to disable tokenizers parallelism warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Konfigurasi umum
num_proc = os.cpu_count()
SAMPLING_RATE = 16000

# Direktori kerja
base_dir = Path("/workspace/data/audio_train/librivox-indonesia")
base_dir.mkdir(parents=True, exist_ok=True)


# Download Dataset Files
This cell downloads the necessary dataset files from Hugging Face, including training and test audio archives, and their corresponding metadata CSV files compressed with gzip.

In [12]:
!wget -nc https://huggingface.co/datasets/indonesian-nlp/librivox-indonesia/resolve/main/data/audio_train.tgz -P /workspace/data/
!wget -nc https://huggingface.co/datasets/indonesian-nlp/librivox-indonesia/resolve/main/data/audio_test.tgz -P /workspace/data/
!wget -nc https://huggingface.co/datasets/indonesian-nlp/librivox-indonesia/resolve/main/data/metadata_train.csv.gz -P /workspace/data/
!wget -nc https://huggingface.co/datasets/indonesian-nlp/librivox-indonesia/resolve/main/data/metadata_test.csv.gz -P /workspace/data/

File ‘/workspace/data/audio_train.tgz’ already there; not retrieving.

File ‘/workspace/data/audio_test.tgz’ already there; not retrieving.

File ‘/workspace/data/metadata_train.csv.gz’ already there; not retrieving.

File ‘/workspace/data/metadata_test.csv.gz’ already there; not retrieving.



# Extract Audio Archives
This cell extracts the downloaded tar.gz archives containing the training and test audio files to the specified directory structure.

In [13]:
# Ekstrak audio
for archive in ["/workspace/data/audio_train.tgz", "/workspace/data/audio_test.tgz"]:
    with tarfile.open(archive, "r:gz") as tar:
        tar.extractall(path="/workspace/data/audio_train")

# Audio Format Conversion
This cell converts all audio files from their original formats to WAV format with mono channel and 16kHz sampling rate using ffmpeg. It processes the files in parallel for efficiency and updates the metadata CSV to reflect the new file paths. The conversion ensures compatibility with the Whisper model's audio processing requirements.

In [14]:
import os
import pandas as pd
import subprocess
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor, as_completed

# --- Paths ---
audio_base_path = "/workspace/data/audio_train/librivox-indonesia"
output_base = os.path.join(audio_base_path, "converted_wav")
os.makedirs(output_base, exist_ok=True)

# 📌 Metadata asli
metadata_path = "/workspace/data/metadata_train.csv.gz"
df = pd.read_csv(metadata_path)

# --- Worker function for parallel ffmpeg ---
def convert_to_wav(rel_path):
    if pd.isna(rel_path):
        return None

    src_path = os.path.join(audio_base_path, rel_path)
    if not os.path.exists(src_path):
        return None

    out_rel_path = os.path.splitext(rel_path)[0] + ".wav"
    out_path = os.path.join(output_base, out_rel_path)
    os.makedirs(os.path.dirname(out_path), exist_ok=True)

    # ✅ Skip jika sudah ada
    if os.path.exists(out_path):
        return os.path.relpath(out_path, audio_base_path)

    cmd = ["ffmpeg", "-y", "-i", src_path, "-ac", "1", "-ar", "16000", out_path]
    subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    return os.path.relpath(out_path, audio_base_path)

# --- Run parallel conversion ---
new_paths = []
with ProcessPoolExecutor(max_workers=os.cpu_count()) as executor:
    futures = {executor.submit(convert_to_wav, rel_path): rel_path for rel_path in df["path"].astype(str)}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Converting to WAV"):
        new_paths.append(future.result())

# --- Update metadata ke versi WAV ---
df["path"] = new_paths
new_meta_path = os.path.join(audio_base_path, "metadata_train_wav.csv")
df.to_csv(new_meta_path, index=False)

print(f"\n✅ Conversion done. Metadata saved: {new_meta_path}")
print(f"📂 Converted audio saved under: {output_base}")

Converting to WAV: 100%|██████████| 7061/7061 [00:10<00:00, 656.05it/s]



✅ Conversion done. Metadata saved: /workspace/data/audio_train/librivox-indonesia/metadata_train_wav.csv
📂 Converted audio saved under: /workspace/data/audio_train/librivox-indonesia/converted_wav


# Cleanup Original Audio Files
This cell removes the original MP3 files after conversion to WAV format to save disk space and avoid confusion between different audio formats.

In [15]:

import glob

def cleanup_mp3(split_dir):
    print(f"🗑️ Cleaning MP3 in {split_dir} ...")
    removed = 0
    for mp3 in glob.glob(os.path.join(split_dir, "**", "*.mp3"), recursive=True):
        try:
            os.remove(mp3)
            removed += 1
        except:
            pass
    print(f"✅ Removed {removed} MP3 files")

cleanup_mp3(audio_base_path)


🗑️ Cleaning MP3 in /workspace/data/audio_train/librivox-indonesia ...
✅ Removed 7815 MP3 files


# Dataset Preparation for Minangkabau Language
This cell loads the converted metadata, filters the dataset to include only Minangkabau language samples, adds absolute file paths, and converts the pandas DataFrame to a Hugging Face Dataset object for model training.

In [16]:
import os
import pandas as pd
import torchaudio
import torch
from datasets import Dataset, Audio
from sklearn.model_selection import train_test_split
from transformers import WhisperProcessor

# --- CONFIGURATION ---
audio_base_path = "/workspace/data/audio_train/librivox-indonesia"
metadata_path = os.path.join(audio_base_path, "metadata_train_wav.csv")
MODEL_NAME = "openai/whisper-small"
TARGET_LANGUAGE = "min"  # Target language code
SAMPLING_RATE = 16000

# --- 1. LOAD METADATA ---
if not os.path.exists(metadata_path):
    raise FileNotFoundError(f"❌ Metadata file not found at {metadata_path}.")

print(f"📂 Loading metadata from: {metadata_path}")
df = pd.read_csv(metadata_path)

# --- 2. CHECK & SELECT LANGUAGE ---
if "language" not in df.columns:
    raise ValueError(f"❌ Column 'language' not found in metadata! Columns: {df.columns}")

print("\n📊 Language Distribution in Dataset:")
print(df["language"].value_counts())

# Filter for Target Language
df_lang = df[df["language"] == TARGET_LANGUAGE].copy()

# AUTO-FALLBACK LOGIC
if len(df_lang) == 0:
    print(f"\n⚠️ WARNING: No samples found for target language '{TARGET_LANGUAGE}'.")
    most_common_lang = df["language"].mode()[0]
    count = len(df[df["language"] == most_common_lang])
    print(f"🔄 Switching to most common language: '{most_common_lang}' ({count} samples)")
    df_lang = df[df["language"] == most_common_lang].copy()
    TARGET_LANGUAGE = most_common_lang

# --- 3. CLEANUP PATHS ---
# Remove NaN paths
df_lang = df_lang.dropna(subset=["path"])
df_lang["path"] = df_lang["path"].astype(str)

# Construct absolute paths
df_lang["full_path"] = df_lang["path"].apply(lambda p: os.path.join(audio_base_path, p))

# Verify files exist (Fast check)
df_lang["exists"] = df_lang["full_path"].apply(os.path.exists)
missing_count = len(df_lang) - df_lang["exists"].sum()
if missing_count > 0:
    print(f"⚠️ Warning: {missing_count} audio files are missing from disk. Dropping them.")
    df_lang = df_lang[df_lang["exists"]]

print(f"✅ Final Dataset Size for '{TARGET_LANGUAGE}': {len(df_lang)} samples")

# --- 4. TRAIN/TEST SPLIT ---
# We do this BEFORE creating the HF Dataset to ensure distinct splits
train_df, val_df = train_test_split(df_lang, test_size=0.1, random_state=42)

train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)

# --- 5. PROCESSOR & FEATURE EXTRACTION ---
print("\n⚙️ Loading Whisper Processor...")
# Note: We use "indonesian" as the language token because Whisper might not support "min" directly
# This helps transfer learning from Indonesian to Minangkabau
lang_token = "indonesian" if TARGET_LANGUAGE in ["min", "ind", "jav", "sun"] else "english"
processor = WhisperProcessor.from_pretrained(MODEL_NAME, language=lang_token, task="transcribe")

def prepare_dataset(batch):
    """Preprocesses audio to input_features"""
    # Load audio
    audio_path = batch["full_path"]
    waveform, sr = torchaudio.load(audio_path)
    
    # Resample if needed
    if sr != SAMPLING_RATE:
        waveform = torchaudio.functional.resample(waveform, sr, SAMPLING_RATE)
    
    # Convert to mono if needed
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)
    
    # Squeeze to 1D array for processor
    waveform = waveform.squeeze()

    # Process audio
    inputs = processor(
        audio=waveform.numpy(),
        sampling_rate=SAMPLING_RATE,
        text=batch["sentence"]
    )
    
    return {
        "input_features": inputs["input_features"][0],
        "labels": inputs["labels"]
    }

print("🔄 Processing audio features (Mapping)...")
# Use num_proc=1 to avoid multiprocessing issues in some notebooks, increase if safe
train_ds = train_ds.map(prepare_dataset, num_proc=1)
val_ds = val_ds.map(prepare_dataset, num_proc=1)

# --- 6. DATA COLLATOR ---
@torch.no_grad()
def data_collator(features):
    input_features = [{"input_features": feature["input_features"]} for feature in features]
    labels = [feature["labels"] for feature in features]

    batch = processor.feature_extractor.pad(input_features, return_tensors="pt")

    # Pad labels manually
    max_label_len = max(len(l) for l in labels)
    padded_labels = []
    
    for label in labels:
        padded_label = label + [-100] * (max_label_len - len(label))
        padded_labels.append(padded_label)
        
    labels_tensor = torch.tensor(padded_labels, dtype=torch.long)
    
    # Trim BOS if present (standard Whisper hygiene)
    if (labels_tensor[:, 0] == processor.tokenizer.bos_token_id).all().cpu().item():
        labels_tensor = labels_tensor[:, 1:]
        
    batch["labels"] = labels_tensor
    return batch

print("\n✅ Data Preparation Complete!")
print(f"   Train set: {len(train_ds)}")
print(f"   Val set:   {len(val_ds)}")
print("   Variables ready: train_ds, val_ds, processor, data_collator")

📂 Loading metadata from: /workspace/data/audio_train/librivox-indonesia/metadata_train_wav.csv

📊 Language Distribution in Dataset:
language
ind    5635
jav     728
bal     159
sun     140
ace     139
min     136
bug     124
Name: count, dtype: int64
✅ Final Dataset Size for 'min': 136 samples

⚙️ Loading Whisper Processor...
🔄 Processing audio features (Mapping)...


Map (num_proc=1):   0%|          | 0/122 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

Map (num_proc=1):   0%|          | 0/14 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r


✅ Data Preparation Complete!
   Train set: 122
   Val set:   14
   Variables ready: train_ds, val_ds, processor, data_collator


# Load Whisper Model and Processor
This cell loads the pre-trained OpenAI Whisper Small model and its processor from Hugging Face. The model is moved to GPU (CUDA) for faster training and inference.

In [17]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

model_name = "openai/whisper-small"
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name)

# Pastikan model ke GPU
model = model.to("cuda")


# Dataset Preprocessing and Training Setup
This cell performs comprehensive setup for fine-tuning the Whisper model: loads the model and processor configured for Minangkabau language transcription, prepares the dataset by splitting into train/validation sets, applies preprocessing to convert audio waveforms to model inputs, and sets up the data collator for sequence-to-sequence training.

In [18]:
import os
import pandas as pd
import torchaudio
from datasets import Dataset, Audio
from transformers import (
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
from sklearn.model_selection import train_test_split
import torch

# --- Config ---
audio_base_path = "/workspace/data/audio_train/librivox-indonesia"
metadata_path = os.path.join(audio_base_path, "metadata_train_wav.csv")
SAMPLING_RATE = 16000
model_name = "openai/whisper-small"

# --- Load model & processor ---
# Change "minangkabau" to "indonesian"
processor = WhisperProcessor.from_pretrained(
    model_name, 
    language="indonesian", 
    task="transcribe"
)
model = WhisperForConditionalGeneration.from_pretrained(model_name)

# Move generation config parameters to avoid warnings
model.generation_config.max_length = 448
model.generation_config.suppress_tokens = [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362]
model.generation_config.begin_suppress_tokens = [220, 50257]

model = model.to("cuda")

# --- STEP 1: Load metadata ---
df = pd.read_csv(metadata_path)
df["full_path"] = df["path"].apply(lambda p: os.path.join(audio_base_path, p))

# Fokus hanya Minangkabau
df = df[df["language"] == "min"].reset_index(drop=True)

# Split train/validation
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

# Convert ke HF Dataset
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)

In [19]:
# Install audiomentations if not present
# !pip install audiomentations

from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, PolarityInversion
import numpy as np

# Define augmentation pipeline
augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
    PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
])

def augment_audio(batch):
    # Load audio using torchaudio
    waveform, sr = torchaudio.load(batch["full_path"])
    
    # Resample if necessary
    if sr != SAMPLING_RATE:
        waveform = torchaudio.functional.resample(waveform, sr, SAMPLING_RATE)
    
    # Convert to numpy for audiomentations
    waveform_np = waveform.numpy()[0] # Take first channel
    
    # Apply augmentation
    augmented_waveform = augment(samples=waveform_np, sample_rate=SAMPLING_RATE)
    
    # Process with Whisper
    inputs = processor(
        audio=augmented_waveform,
        sampling_rate=SAMPLING_RATE,
        text=batch["sentence"]
    )
    
    return {
        "input_features": inputs["input_features"][0],
        "labels": inputs["labels"]
    }

# Apply augmentation ONLY to the training set
# Note: This applies augmentation once per epoch. 
# For better results, apply it dynamically inside a custom DataCollator, 
# but this `map` method is a good starting point.
train_ds = train_ds.map(augment_audio, num_proc=1)

Map (num_proc=1):   0%|          | 0/122 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

In [20]:
# --- STEP 2: Preprocessing function ---
def prepare_dataset(batch):
    # load audio
    waveform, sr = torchaudio.load(batch["full_path"])
    if sr != SAMPLING_RATE:
        waveform = torchaudio.functional.resample(waveform, sr, SAMPLING_RATE)
    waveform = waveform.mean(dim=0)  # mono

    # process
    inputs = processor(
        audio=waveform.numpy(),
        sampling_rate=SAMPLING_RATE,
        text=batch["sentence"]
    )
    return {
        "input_features": inputs["input_features"][0],
        "labels": inputs["labels"]
    }

train_ds = train_ds.map(prepare_dataset)
val_ds = val_ds.map(prepare_dataset)

Map:   0%|          | 0/122 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

In [21]:
import torch

# --- STEP 3: Data Collator (fixed for tokenized labels) ---
@torch.no_grad()
def data_collator(features):
    # Separate inputs and labels for clarity
    input_features = [{"input_features": feature["input_features"]} for feature in features]
    labels = [feature["labels"] for feature in features]

    # Use feature extractor for audio inputs
    batch = processor.feature_extractor.pad(input_features, return_tensors="pt")

    # Convert labels to tensors and pad them manually
    # Labels are already tokenized, so we need to pad them as integers
    max_length = max(len(label) for label in labels)
    
    padded_labels = []
    attention_masks = []
    
    for label in labels:
        # Pad with -100 (ignored in loss calculation)
        padded_label = label + [-100] * (max_length - len(label))
        attention_mask = [1] * len(label) + [0] * (max_length - len(label))
        
        padded_labels.append(padded_label)
        attention_masks.append(attention_mask)
    
    # Convert to tensors
    labels_tensor = torch.tensor(padded_labels, dtype=torch.long)
    attention_mask = torch.tensor(attention_masks, dtype=torch.long)
    
    # Replace padding tokens with -100 for loss calculation
    labels_tensor = labels_tensor.masked_fill(attention_mask.ne(1), -100)

    # Trim BOS token if present (crucial for Whisper)
    if (labels_tensor[:, 0] == processor.tokenizer.bos_token_id).all().cpu().item():
        labels_tensor = labels_tensor[:, 1:]

    batch["labels"] = labels_tensor

    return batch

# Model Training Execution
This cell initiates the fine-tuning process of the Whisper model on the Minangkabau language dataset using the configured trainer. The training will run for the specified number of epochs with periodic evaluation and checkpoint saving.

In [22]:
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
import gc
import evaluate

# --- STEP 1: Define Metrics ---
# Load the WER metric
metric = evaluate.load("wer")

def compute_metrics(pred):
    """
    Computes Word Error Rate (WER) for evaluation.
    """
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Replace -100 in the labels with the pad_token_id (so we can decode them)
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # Decode predictions and references
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    # Compute WER
    # We multiply by 100 to make it easier to read (e.g., 15.5 instead of 0.155)
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

# --- STEP 2: Clean Memory ---
# Delete previous instances to clear VRAM and computation graph
if 'trainer' in globals(): del trainer
if 'model' in globals(): del model
gc.collect()
torch.cuda.empty_cache()

# --- STEP 3: Reload Model (Clean Slate) ---
print("🚀 Loading fresh model on NVIDIA A40...")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.to("cuda")

# Configure Model
# Disable cache for training (prevents warnings)
model.config.use_cache = False 
# IMPORTANT: We DO NOT enable gradient checkpointing because you have an A40 (48GB VRAM)
# and the model is small. Enabling it caused the previous RuntimeError.

# --- STEP 4: Training Arguments (Optimized for A40) ---
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-minang-checkpoints",
    
    # POWER MOVE: Increasing batch size because you have 48GB VRAM
    per_device_train_batch_size=32,  # Increased from 8 -> 32 for speed
    gradient_accumulation_steps=1,   # No need to accumulate with high batch size
    
    learning_rate=1e-5,              # Low LR for stability
    warmup_steps=50,
    max_steps=1000,                  # Force 1000 update steps
    
    gradient_checkpointing=False,    # <--- CRITICAL: DISABLED to fix RuntimeError
    
    fp16=True,                       # A40 handles FP16 perfectly
    eval_strategy="steps",           # Renamed from evaluation_strategy
    eval_steps=100,
    save_steps=100,
    logging_steps=25,
    report_to="tensorboard",
    predict_with_generate=True,
    generation_max_length=225,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
)

# --- STEP 5: Initialize Trainer ---
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=data_collator,
    processing_class=processor,
    compute_metrics=compute_metrics,  # <--- CRITICAL FIX: This was missing in your code!
)

# --- STEP 6: Start Training ---
print("✅ Fresh model loaded. Gradient Checkpointing DISABLED. Metric function ADDED.")
print("🚀 Starting training...")
trainer.train()

🚀 Loading fresh model on NVIDIA A40...
✅ Fresh model loaded. Gradient Checkpointing DISABLED. Metric function ADDED.
🚀 Starting training...


Step,Training Loss,Validation Loss,Wer
100,0.851200,4.386569,145.853659
200,0.007800,4.484226,257.073171
300,0.000600,4.643561,177.560976
400,0.000400,4.675807,184.878049
500,0.000300,4.683607,185.853659
600,0.000300,4.689660,180.000000
700,0.000300,4.695017,180.000000
800,0.000300,4.700410,177.560976
900,0.000200,4.703907,176.097561
1000,0.000200,4.705173,176.097561


Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61

TrainOutput(global_step=1000, training_loss=0.4096466833022423, metrics={'train_runtime': 3728.871, 'train_samples_per_second': 8.582, 'train_steps_per_second': 0.268, 'total_flos': 8.80185470976e+18, 'train_loss': 0.4096466833022423, 'epoch': 250.0})

# Model Testing and Evaluation
This section loads the test dataset and evaluates the fine-tuned Whisper model's performance on unseen Minangkabau audio samples. It processes the test audio files and compares the model's transcriptions with the ground truth.

In [23]:
# --- Prepare Test Data ---
import os
import pandas as pd
import tarfile

# Extract test audio if not already extracted
test_archive = "/workspace/data/audio_test.tgz"
if os.path.exists(test_archive):
    print("📂 Extracting test audio files...")
    with tarfile.open(test_archive, "r:gz") as tar:
        tar.extractall(path="/workspace/data/audio_train")
    print("✅ Test audio extracted")

# Load test metadata
test_metadata_path = "/workspace/data/metadata_test.csv.gz"
test_df = pd.read_csv(test_metadata_path)

# Filter for Minangkabau language only
test_df_min = test_df[test_df["language"] == "min"].reset_index(drop=True)
print(f"📊 Total test samples: {len(test_df)}")
print(f"🎯 Minangkabau test samples: {len(test_df_min)}")

# Add full paths
audio_base_path = "/workspace/data/audio_train/librivox-indonesia"
test_df_min["full_path"] = test_df_min["path"].apply(lambda p: os.path.join(audio_base_path, p))

# Show sample
print(f"\n📝 Sample test data:")
print(test_df_min[["sentence", "path"]].head(3))

📂 Extracting test audio files...
✅ Test audio extracted
📊 Total test samples: 754
🎯 Minangkabau test samples: 20

📝 Sample test data:
                                            sentence  \
0                       katahui bana hak hak awak ko   
1  dan kabebasan dari raso takuik dan dari kakura...   
2                           supayo manjadi kanyataan   

                                                path  
0  test/minangkabau/universal-declaration-of-huma...  
1  test/minangkabau/universal-declaration-of-huma...  
2  test/minangkabau/universal-declaration-of-huma...  


In [24]:
# --- Convert Test Audio to WAV (if needed) ---
import subprocess
from tqdm import tqdm

def convert_test_audio_to_wav(df, output_base):
    """Convert test audio files to WAV format for consistency"""
    os.makedirs(output_base, exist_ok=True)
    
    converted_paths = []
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Converting test audio"):
        rel_path = row["path"]
        src_path = os.path.join(audio_base_path, rel_path)
        
        # Skip if source doesn't exist
        if not os.path.exists(src_path):
            converted_paths.append(None)
            continue
            
        # Create WAV output path
        out_rel_path = os.path.splitext(rel_path)[0] + ".wav"
        out_path = os.path.join(output_base, out_rel_path)
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        
        # Skip if already converted
        if os.path.exists(out_path):
            converted_paths.append(os.path.relpath(out_path, audio_base_path))
            continue
            
        # Convert to WAV
        cmd = ["ffmpeg", "-y", "-i", src_path, "-ac", "1", "-ar", "16000", out_path]
        result = subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        
        if result.returncode == 0:
            converted_paths.append(os.path.relpath(out_path, audio_base_path))
        else:
            converted_paths.append(None)
    
    return converted_paths

# Convert test audio
test_output_base = os.path.join(audio_base_path, "converted_wav")
test_df_min["wav_path"] = convert_test_audio_to_wav(test_df_min, test_output_base)
test_df_min["wav_full_path"] = test_df_min["wav_path"].apply(
    lambda p: os.path.join(audio_base_path, p) if p else None
)

# Remove rows where conversion failed
test_df_clean = test_df_min.dropna(subset=["wav_full_path"]).reset_index(drop=True)
print(f"✅ Successfully converted {len(test_df_clean)} test audio files")

Converting test audio: 100%|██████████| 20/20 [00:06<00:00,  2.97it/s]

✅ Successfully converted 20 test audio files


In [25]:
# --- Test Model Inference ---
import torchaudio
from transformers import pipeline

def transcribe_audio(audio_path, model, processor):
    """Transcribe a single audio file using the fine-tuned model"""
    try:
        # Load audio
        waveform, sr = torchaudio.load(audio_path)
        if sr != SAMPLING_RATE:
            waveform = torchaudio.functional.resample(waveform, sr, SAMPLING_RATE)
        waveform = waveform.mean(dim=0)  # Convert to mono
        
        # Process audio
        inputs = processor(
            audio=waveform.numpy(),
            sampling_rate=SAMPLING_RATE,
            return_tensors="pt"
        )
        
        # Move to GPU
        inputs = {k: v.to("cuda") for k, v in inputs.items()}
        
        # Generate transcription
        with torch.no_grad():
            generated_ids = model.generate(
                inputs["input_features"],
                max_length=448,
                num_beams=1,
                do_sample=False
            )
        
        # Decode transcription
        transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        return transcription
        
    except Exception as e:
        print(f"Error transcribing {audio_path}: {e}")
        return None

# Test on a small sample first (5 files)
test_sample = test_df_clean.head(20).copy()
print("🎤 Testing model on sample audio files...")

predictions = []
ground_truths = []

for idx, row in test_sample.iterrows():
    print(f"Processing {idx+1}/20: {os.path.basename(row['wav_full_path'])}")
    
    # Get prediction
    prediction = transcribe_audio(row["wav_full_path"], model, processor)
    ground_truth = row["sentence"]
    
    predictions.append(prediction)
    ground_truths.append(ground_truth)
    
    print(f"  Ground Truth: {ground_truth}")
    print(f"  Prediction:   {prediction}")
    print("  " + "="*50)

print(f"\n✅ Completed testing on {len(test_sample)} samples")

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

🎤 Testing model on sample audio files...
Processing 1/20: human_rights_un_min_sd_0009.wav
  Ground Truth: katahui bana hak hak awak ko
  Prediction:    وقت التحديث عن المدينة
Processing 2/20: human_rights_un_min_sd_0016.wav
  Ground Truth: dan kabebasan dari raso takuik dan dari kakurangan
  Prediction:   dan kalaki nan parangakiranjan ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki
Processing 3/20: human_rights_un_min_sd_0028.wav
  Ground Truth: supayo manjadi kanyataan
  Prediction:   akimajal niyo
Processing 4/20: hum

In [26]:
# --- Calculate Evaluation Metrics ---
try:
    import jiwer
    
    # Filter out None predictions
    valid_pairs = [(pred, gt) for pred, gt in zip(predictions, ground_truths) if pred is not None]
    
    if valid_pairs:
        valid_predictions, valid_ground_truths = zip(*valid_pairs)
        
        # Convert tuples to lists for jiwer
        valid_predictions = list(valid_predictions)
        valid_ground_truths = list(valid_ground_truths)
        
        # Calculate Word Error Rate (WER)
        wer_score = jiwer.wer(valid_ground_truths, valid_predictions)
        
        # Calculate Character Error Rate (CER)
        cer_score = jiwer.cer(valid_ground_truths, valid_predictions)
        
        print("📊 EVALUATION RESULTS:")
        print("="*50)
        print(f"📝 Number of samples: {len(valid_pairs)}")
        print(f"🎯 Word Error Rate (WER): {wer_score:.4f} ({wer_score*100:.2f}%)")
        print(f"🔤 Character Error Rate (CER): {cer_score:.4f} ({cer_score*100:.2f}%)")
        print(f"✅ Word Accuracy: {(1-wer_score)*100:.2f}%")
        
        # Show detailed comparison
        print(f"\n📋 DETAILED COMPARISON:")
        print("="*50)
        for i, (pred, gt) in enumerate(valid_pairs):
            print(f"Sample {i+1}:")
            print(f"  Ground Truth: {gt}")
            print(f"  Prediction:   {pred}")
            
            # Calculate individual metrics
            individual_wer = jiwer.wer([gt], [pred])
            individual_cer = jiwer.cer([gt], [pred])
            print(f"  WER: {individual_wer:.4f}, CER: {individual_cer:.4f}")
            print()
            
    else:
        print("❌ No valid predictions to evaluate")
        
except ImportError:
    print("⚠️ jiwer not available. Install with: pip install jiwer")
    print("Showing basic comparison instead...")
    
    for i, (pred, gt) in enumerate(zip(predictions, ground_truths)):
        if pred is not None:
            print(f"Sample {i+1}:")
            print(f"  Ground Truth: {gt}")
            print(f"  Prediction:   {pred}")
            print(f"  Match: {'✅' if pred.lower().strip() == gt.lower().strip() else '❌'}")
            print()
            
except Exception as e:
    print(f"⚠️ Error calculating metrics: {e}")
    print("Showing basic comparison instead...")
    
    for i, (pred, gt) in enumerate(zip(predictions, ground_truths)):
        if pred is not None:
            print(f"Sample {i+1}:")
            print(f"  Ground Truth: {gt}")
            print(f"  Prediction:   {pred}")
            print(f"  Match: {'✅' if pred.lower().strip() == gt.lower().strip() else '❌'}")
            print()

📊 EVALUATION RESULTS:
📝 Number of samples: 20
🎯 Word Error Rate (WER): 3.8238 (382.38%)
🔤 Character Error Rate (CER): 3.4872 (348.72%)
✅ Word Accuracy: -282.38%

📋 DETAILED COMPARISON:
Sample 1:
  Ground Truth: katahui bana hak hak awak ko
  Prediction:    وقت التحديث عن المدينة
  WER: 1.0000, CER: 0.8929

Sample 2:
  Ground Truth: dan kabebasan dari raso takuik dan dari kakurangan
  Prediction:   dan kalaki nan parangakiranjan ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki ौpaki
  WER: 11.2500, CER: 10.4400

Sample 3:
  Grou

In [27]:
# --- Improved Model Testing with Better Generation Parameters ---
def transcribe_audio_improved(audio_path, model, processor):
    """Transcribe with improved generation parameters to avoid repetition"""
    try:
        # Load audio
        waveform, sr = torchaudio.load(audio_path)
        if sr != SAMPLING_RATE:
            waveform = torchaudio.functional.resample(waveform, sr, SAMPLING_RATE)
        waveform = waveform.mean(dim=0)  # Convert to mono
        
        # Process audio
        inputs = processor(
            audio=waveform.numpy(),
            sampling_rate=SAMPLING_RATE,
            return_tensors="pt"
        )
        
        # Move to GPU
        inputs = {k: v.to("cuda") for k, v in inputs.items()}
        
        # Generate transcription with improved parameters
        with torch.no_grad():
            generated_ids = model.generate(
                inputs["input_features"],
                max_length=200,  # Reduced from 448
                min_length=1,
                num_beams=3,     # Increased from 1
                do_sample=False,
                early_stopping=True,
                no_repeat_ngram_size=3,  # Prevent repetition
                repetition_penalty=1.2,   # Penalize repetition
                length_penalty=1.0,
                bad_words_ids=[[50257]],  # Avoid problematic tokens
                forced_decoder_ids=None,
                temperature=1.0
            )
        
        # Decode transcription
        transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        return transcription.strip()
        
    except Exception as e:
        print(f"Error transcribing {audio_path}: {e}")
        return None

# Test with improved parameters on first 5 samples only
test_sample_improved = test_df_clean.head(5).copy()
print("🎤 Re-testing model with improved generation parameters...")

improved_predictions = []
improved_ground_truths = []

for idx, row in test_sample_improved.iterrows():
    print(f"Processing {idx+1}/5: {os.path.basename(row['wav_full_path'])}")
    
    # Get prediction with improved parameters
    prediction = transcribe_audio_improved(row["wav_full_path"], model, processor)
    ground_truth = row["sentence"]
    
    improved_predictions.append(prediction)
    improved_ground_truths.append(ground_truth)
    
    print(f"  Ground Truth: {ground_truth}")
    print(f"  Prediction:   {prediction}")
    print("  " + "="*50)

print(f"\n✅ Completed improved testing on {len(test_sample_improved)} samples")

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🎤 Re-testing model with improved generation parameters...
Processing 1/5: human_rights_un_min_sd_0009.wav
  Ground Truth: katahui bana hak hak awak ko
  Prediction:   kaluarga atau manusia atau malakukan di tiok urang sa kadalamuran daripadaan iduik nan basawakai?
Processing 2/5: human_rights_un_min_sd_0016.wav
  Ground Truth: dan kabebasan dari raso takuik dan dari kakurangan
  Prediction:   ka民ana ireksi tiok urang punyo hak mardeka untuak nafkah jinih parkawinan nan damai ka khusambagian
Processing 3/5: human_rights_un_min_sd_0028.wav
  Ground Truth: supayo manjadi kanyataan
  Prediction:   man sadism anna baiak kaya negaraan jo randunnan musti pasatudok manyukaienho
Processing 4/5: human_rights_un_min_sd_0039.wav
  Ground Truth: indak ado pambedaan   umpamonyo pambedaan ras
  Prediction:   sakai jo pandiliranyo sabagai?
Processing 5/5: human_rights_un_min_sd_0044.wav
  Ground Truth: indak diadokan pambedaan badasar kadudukan politik
  Prediction:   pasal dua belas tiok urang punyo 

In [28]:
# --- Calculate Improved Evaluation Metrics ---
try:
    import jiwer
    
    # Filter out None predictions from improved results
    valid_pairs_improved = [(pred, gt) for pred, gt in zip(improved_predictions, improved_ground_truths) if pred is not None and pred.strip() != ""]
    
    if valid_pairs_improved:
        valid_predictions_improved, valid_ground_truths_improved = zip(*valid_pairs_improved)
        
        # Convert tuples to lists for jiwer
        valid_predictions_improved = list(valid_predictions_improved)
        valid_ground_truths_improved = list(valid_ground_truths_improved)
        
        # Calculate Word Error Rate (WER)
        wer_score_improved = jiwer.wer(valid_ground_truths_improved, valid_predictions_improved)
        
        # Calculate Character Error Rate (CER)
        cer_score_improved = jiwer.cer(valid_ground_truths_improved, valid_predictions_improved)
        
        print("📊 IMPROVED EVALUATION RESULTS:")
        print("="*60)
        print(f"📝 Number of valid samples: {len(valid_pairs_improved)}")
        print(f"🎯 Word Error Rate (WER): {wer_score_improved:.4f} ({wer_score_improved*100:.2f}%)")
        print(f"🔤 Character Error Rate (CER): {cer_score_improved:.4f} ({cer_score_improved*100:.2f}%)")
        print(f"✅ Word Accuracy: {(1-wer_score_improved)*100:.2f}%")
        print(f"📈 Character Accuracy: {(1-cer_score_improved)*100:.2f}%")
        
        # Show detailed comparison
        print(f"\n📋 DETAILED COMPARISON (IMPROVED):")
        print("="*60)
        for i, (pred, gt) in enumerate(valid_pairs_improved):
            print(f"Sample {i+1}:")
            print(f"  Ground Truth: {gt}")
            print(f"  Prediction:   {pred}")
            
            # Calculate individual metrics
            individual_wer = jiwer.wer([gt], [pred])
            individual_cer = jiwer.cer([gt], [pred])
            print(f"  📊 WER: {individual_wer:.4f}, CER: {individual_cer:.4f}")
            
            # Show if it's a good prediction
            if individual_wer < 0.3:
                print(f"  ✅ Good transcription!")
            elif individual_wer < 0.6:
                print(f"  ⚠️ Moderate accuracy")
            else:
                print(f"  ❌ Needs improvement")
            print()
            
    else:
        print("❌ No valid predictions to evaluate in improved results")
        
except Exception as e:
    print(f"⚠️ Error calculating improved metrics: {e}")
    print("Showing basic comparison instead...")
    
    for i, (pred, gt) in enumerate(zip(improved_predictions, improved_ground_truths)):
        if pred is not None and pred.strip() != "":
            print(f"Sample {i+1}:")
            print(f"  Ground Truth: {gt}")
            print(f"  Prediction:   {pred}")
            print(f"  Match: {'✅' if pred.lower().strip() == gt.lower().strip() else '❌'}")
            print()

📊 IMPROVED EVALUATION RESULTS:
📝 Number of valid samples: 5
🎯 Word Error Rate (WER): 1.9310 (193.10%)
🔤 Character Error Rate (CER): 1.4112 (141.12%)
✅ Word Accuracy: -93.10%
📈 Character Accuracy: -41.12%

📋 DETAILED COMPARISON (IMPROVED):
Sample 1:
  Ground Truth: katahui bana hak hak awak ko
  Prediction:   kaluarga atau manusia atau malakukan di tiok urang sa kadalamuran daripadaan iduik nan basawakai?
  📊 WER: 2.3333, CER: 2.7143
  ❌ Needs improvement

Sample 2:
  Ground Truth: dan kabebasan dari raso takuik dan dari kakurangan
  Prediction:   ka民ana ireksi tiok urang punyo hak mardeka untuak nafkah jinih parkawinan nan damai ka khusambagian
  📊 WER: 1.8750, CER: 1.3600
  ❌ Needs improvement

Sample 3:
  Ground Truth: supayo manjadi kanyataan
  Prediction:   man sadism anna baiak kaya negaraan jo randunnan musti pasatudok manyukaienho
  📊 WER: 3.6667, CER: 2.5000
  ❌ Needs improvement

Sample 4:
  Ground Truth: indak ado pambedaan   umpamonyo pambedaan ras
  Prediction:   sakai jo p

# Model Analysis and Improvement Suggestions
This section analyzes the model performance and provides insights into potential improvements. The results show that while the model has learned some Minangkabau patterns, it needs more training or different hyperparameters to achieve better accuracy.

In [29]:
# --- Performance Analysis ---
print("🔍 PERFORMANCE ANALYSIS:")
print("="*60)

# Calculate basic statistics from the improved results
if 'valid_pairs_improved' in locals() and valid_pairs_improved:
    print(f"\n📊 Current Model Performance:")
    print(f"   • Word Error Rate: {wer_score_improved*100:.1f}%")
    print(f"   • Character Error Rate: {cer_score_improved*100:.1f}%") 
    print(f"   • Word Accuracy: {(1-wer_score_improved)*100:.1f}%")
    print(f"   • Character Accuracy: {(1-cer_score_improved)*100:.1f}%")
    
    print(f"\n🎯 Accuracy Interpretation:")
    if wer_score_improved > 0.8:
        print("   ❌ High WER (>80%): Model needs significant improvement")
    elif wer_score_improved > 0.5:
        print("   ⚠️ Moderate WER (50-80%): Model shows some learning but needs refinement")
    elif wer_score_improved > 0.3:
        print("   🔶 Fair WER (30-50%): Model is learning patterns well")
    else:
        print("   ✅ Good WER (<30%): Model performing well")
        
    print(f"\n🔤 Character-level Analysis:")
    if cer_score_improved < 0.3:
        print("   ✅ Good character accuracy - model understands phonetics")
    elif cer_score_improved < 0.6:
        print("   🔶 Moderate character accuracy - some phonetic understanding")
    else:
        print("   ❌ Poor character accuracy - limited phonetic learning")

print(f"\n📈 Training Insights:")
print("   • Short training (3 epochs, 12 steps) - likely underfitted")
print("   • Model shows some Minangkabau patterns but needs more exposure")
print("   • Character-level performance better than word-level suggests partial learning")

print(f"\n🛠️ Improvement Strategies:")
print("   1. Increase training epochs (5-10 epochs)")
print("   2. Lower learning rate for fine-grained learning")
print("   3. Increase dataset size if possible")
print("   4. Use data augmentation (speed/pitch variations)")
print("   5. Fine-tune generation parameters further")
print("   6. Consider using Whisper-base instead of small for better capacity")

🔍 PERFORMANCE ANALYSIS:

📊 Current Model Performance:
   • Word Error Rate: 193.1%
   • Character Error Rate: 141.1%
   • Word Accuracy: -93.1%
   • Character Accuracy: -41.1%

🎯 Accuracy Interpretation:
   ❌ High WER (>80%): Model needs significant improvement

🔤 Character-level Analysis:
   ❌ Poor character accuracy - limited phonetic learning

📈 Training Insights:
   • Short training (3 epochs, 12 steps) - likely underfitted
   • Model shows some Minangkabau patterns but needs more exposure
   • Character-level performance better than word-level suggests partial learning

🛠️ Improvement Strategies:
   1. Increase training epochs (5-10 epochs)
   2. Lower learning rate for fine-grained learning
   3. Increase dataset size if possible
   4. Use data augmentation (speed/pitch variations)
   5. Fine-tune generation parameters further
   6. Consider using Whisper-base instead of small for better capacity


In [30]:
# --- Save Model for Future Use ---
print("💾 SAVING FINE-TUNED MODEL:")
print("="*50)

try:
    # Save the fine-tuned model and processor
    model_save_path = "./whisper-minang-final"
    
    print(f"Saving model to: {model_save_path}")
    model.save_pretrained(model_save_path)
    processor.save_pretrained(model_save_path)
    
    print("✅ Model and processor saved successfully!")
    print(f"📁 Location: {model_save_path}")
    
    # Save a summary of the training results
    results_summary = {
        "model_name": "whisper-small-minangkabau",
        "training_epochs": 3,
        "training_steps": 12,
        "final_loss": 5.78,
        "test_samples": len(valid_pairs_improved) if 'valid_pairs_improved' in locals() else 0,
        "word_error_rate": f"{wer_score_improved*100:.2f}%" if 'wer_score_improved' in locals() else "N/A",
        "character_error_rate": f"{cer_score_improved*100:.2f}%" if 'cer_score_improved' in locals() else "N/A",
        "word_accuracy": f"{(1-wer_score_improved)*100:.2f}%" if 'wer_score_improved' in locals() else "N/A",
        "character_accuracy": f"{(1-cer_score_improved)*100:.2f}%" if 'cer_score_improved' in locals() else "N/A"
    }
    
    import json
    with open(f"{model_save_path}/training_results.json", "w") as f:
        json.dump(results_summary, f, indent=2)
    
    print("📊 Training results summary saved!")
    
    # Instructions for loading the model later
    print(f"\n🔄 To load this model later, use:")
    print(f"```python")
    print(f"from transformers import WhisperProcessor, WhisperForConditionalGeneration")
    print(f"processor = WhisperProcessor.from_pretrained('{model_save_path}')")
    print(f"model = WhisperForConditionalGeneration.from_pretrained('{model_save_path}')")
    print(f"```")
    
except Exception as e:
    print(f"❌ Error saving model: {e}")
    print("The model is still available in memory for this session.")

💾 SAVING FINE-TUNED MODEL:
Saving model to: ./whisper-minang-final
✅ Model and processor saved successfully!
📁 Location: ./whisper-minang-final
📊 Training results summary saved!

🔄 To load this model later, use:
```python
from transformers import WhisperProcessor, WhisperForConditionalGeneration
processor = WhisperProcessor.from_pretrained('./whisper-minang-final')
model = WhisperForConditionalGeneration.from_pretrained('./whisper-minang-final')
```
